# Additional training functions

`train` provides a number of extension methods that are added to `Learner` (see below for a list and details), along with three simple callbacks:

- `ShowGraph`
- `GradientClipping`
- `BnFreeze`

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.train import * 

## `Learner` extension methods

These methods are automatically added to all `Learner` objects created after importing this module. They provide convenient access to various callbacks, without requiring them to be manually created.

In [ ]:
show_doc(fit_one_cycle)

#### <a id=fit_one_cycle></a>`fit_one_cycle`
`fit_one_cycle`(<code>learn</code>:[<code>Learner</code>](/basic_train.html#Learner), <code>cyc_len</code>:<code>int</code>, <code>max_lr</code>:<code>Union</code>[<code>float</code>, <code>Collection</code>[<code>float</code>], <code>slice</code>]=`slice(None, 0.003, None)`, <code>moms</code>:<code>Tuple</code>[<code>float</code>, <code>float</code>]=`(0.95, 0.85)`, <code>div_factor</code>:<code>float</code>=`25.0`, <code>pct_start</code>:<code>float</code>=`0.3`, <code>wd</code>:<code>float</code>=`None`, <code>kwargs</code>)


Fits a model following the 1cycle policy <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L11">[source]</a>

In [ ]:
from fastai.callbacks import *

In [ ]:
show_doc(lr_find)

#### <a id=lr_find></a>`lr_find`
`lr_find`(<code>learn</code>:[<code>Learner</code>](/basic_train.html#Learner), <code>start_lr</code>:<code>float</code>=`1e-05`, <code>end_lr</code>:<code>float</code>=`10`, <code>num_it</code>:<code>int</code>=`100`, <code>kwargs</code>:<code>Any</code>)


Explore lr from `start_lr` to `end_lr` over `num_it` iterations of `learn` <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L20">[source]</a>

See `OneCycleScheduler` for details.

In [ ]:
show_doc(one_cycle_scheduler)

#### <a id=one_cycle_scheduler></a>`one_cycle_scheduler`
`one_cycle_scheduler`(<code>lr_max</code>:<code>float</code>, <code>kwargs</code>:<code>Any</code>) -> [<code>OneCycleScheduler</code>](/callbacks.one_cycle.html#OneCycleScheduler)
<a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L8">[source]</a>

See `OneCycleScheduler` for details.

In [ ]:
show_doc(to_fp16)

#### <a id=to_fp16></a>`to_fp16`
`to_fp16`(<code>learn</code>:[<code>Learner</code>](/basic_train.html#Learner), <code>loss_scale</code>:<code>float</code>=`512.0`, <code>flat_master</code>:<code>bool</code>=`False`) -> [<code>Learner</code>](/basic_train.html#Learner)


Transforms the learner in FP16 precision <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L26">[source]</a>

See `OneCycleScheduler` for details.

In [ ]:
show_doc(ShowGraph)

## <a id=ShowGraph></a>class `ShowGraph`
`ShowGraph`(<code>learn</code>:[<code>Learner</code>](/basic_train.html#Learner)) :: [<code>LearnerCallback</code>](/basic_train.html#LearnerCallback)


Updates a graph of learner stats and metrics after each epoch <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L44">[source]</a>

[<code>ShowGraph</code>](/train.html#ShowGraph)

### Internal callback implementation

In [ ]:
show_doc(ShowGraph.on_epoch_end)

#### <a id=on_epoch_end></a>`on_epoch_end`
`on_epoch_end`(<code>n_epochs</code>:<code>int</code>, <code>last_metrics</code>:<code>Collection</code>[<code>Union</code>[<code>Tensor</code>, <code>Number</code>]], <code>kwargs</code>) -> <code>bool</code>


If we have metrics plot them in our pbar graph <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L46">[source]</a>

In [ ]:
show_doc(GradientClipping)

## <a id=GradientClipping></a>class `GradientClipping`
`GradientClipping`(<code>learn</code>:[<code>Learner</code>](/basic_train.html#Learner), <code>clip</code>:<code>float</code>) :: [<code>LearnerCallback</code>](/basic_train.html#LearnerCallback)


To do gradient clipping during training. <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L64">[source]</a>

[<code>GradientClipping</code>](/train.html#GradientClipping)

### Internal callback implementation

In [ ]:
show_doc(GradientClipping.on_backward_end)

#### <a id=on_backward_end></a>`on_backward_end`
`on_backward_end`(<code>kwargs</code>)


Called after backprop but before optimizer step. Useful for true weight decay in AdamW <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L68">[source]</a>

In [ ]:
show_doc(BnFreeze)

## <a id=BnFreeze></a>class `BnFreeze`
`BnFreeze`(<code>learn</code>:[<code>Learner</code>](/basic_train.html#Learner)) :: [<code>LearnerCallback</code>](/basic_train.html#LearnerCallback)


Set all bntypes layers in `learn` to eval() on_epoch_begin <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L57">[source]</a>

[<code>BnFreeze</code>](/train.html#BnFreeze)

### Internal callback implementation

In [ ]:
show_doc(BnFreeze.on_epoch_begin)

#### <a id=on_epoch_begin></a>`on_epoch_begin`
`on_epoch_begin`(<code>kwargs</code>:<code>Any</code>)


Put bn layers in eval mode on epoch_begin <a href="https://github.com/fastai/fastai_pytorch/blob/master/fastai/train.py#L59">[source]</a>